In [1]:
import numpy as np
import torch
from torchvision.datasets import mnist
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [2]:
import time
batch_size=256
test_dataset = mnist.MNIST(root='../../test', train=False, transform=ToTensor())
test_loader = DataLoader(test_dataset, batch_size=batch_size)
def calc():
    all_correct_num = 0
    all_sample_num = 0
    model.eval()
    t_ls=[]
    fps_ls=[]
    for idx, (test_x, test_label) in enumerate(test_loader):
        start = time.time()
        predict_y = model(test_x.float()).detach()
        end = time.time()
        total_time = (end - start)
        if total_time!=0:
            fps=1/(total_time)
            fps_ls.append(fps)
        t_ls.append(total_time)
        predict_y = np.argmax(predict_y, axis=-1)
        current_correct_num = predict_y == test_label
        all_correct_num += np.sum(current_correct_num.numpy(), axis=-1)
        all_sample_num += current_correct_num.shape[0]
    t=sum(t_ls)
    fps=sum(fps_ls)/len(fps_ls)
    avg_time=t/len(t_ls)
    acc = all_correct_num / all_sample_num
    return t,avg_time,fps,acc

In [8]:
model=torch.load("mnist_0.98130.pkl")
calc()

(0.18804407119750977, 0.004701101779937744, 229.94763159564596, 0.9813)

In [7]:
from torchsummary import summary
summary(model,(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 5, 22, 22]             250
              ReLU-2            [-1, 5, 22, 22]               0
         MaxPool2d-3            [-1, 5, 11, 11]               0
            Conv2d-4              [-1, 7, 7, 7]             882
              ReLU-5              [-1, 7, 7, 7]               0
         MaxPool2d-6              [-1, 7, 3, 3]               0
            Linear-7                   [-1, 15]             960
              ReLU-8                   [-1, 15]               0
            Linear-9                   [-1, 10]             160
             ReLU-10                   [-1, 10]               0
Total params: 2,252
Trainable params: 2,252
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.01
Estimated Total

In [21]:
s=0
a=0
for i in range(100):
    temp=calc()
    s+=temp[1]
    a+=temp[2]
print(s/100,a/100)

5304.704833493355 0.988399999999999
